<a href="https://colab.research.google.com/github/NajimovOtabek/net_intrusion_detection/blob/master/Comparing_otimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
!unzip MachineLearningCSV.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
To: /content/MachineLearningCSV.zip
100% 235M/235M [00:03<00:00, 74.6MB/s]
Archive:  MachineLearningCSV.zip
   creating: MachineLearningCVE/
  inflating: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  i

In [2]:
!wget https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py

--2023-08-23 02:22:09--  https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3790 (3.7K) [text/plain]
Saving to: ‘preprocessing.py’

preprocessing.py    100%[===================>]   3.70K  --.-KB/s    in 0s      

2023-08-23 02:22:10 (45.6 MB/s) - ‘preprocessing.py’ saved [3790/3790]



In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score, classification_report
from preprocessing import read_data, load_data, normalize
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import os

In [4]:
dataroot = 'MachineLearningCVE/'
from preprocessing import read_data
data = read_data(dataroot,'*.pcap_ISCX.csv')

[########################################] | 100% Completed | 28.24 s


In [5]:
# Load and normalize the data
X, y = load_data(dataroot)
X = normalize(X)

[########################################] | 100% Completed | 28.11 s
there are 2830743 flow records with 79 feature dimension
stripped column names
dropped bad columns
There are 0 nan entries
converted to numeric


In [6]:
def stratified_split(X, y, test_size=0.1, random_state=42, min_class_count=2000):
    # Identify classes with at least min_class_count instances
    counts = Counter(y)
    valid_classes = [cls for cls, count in counts.items() if count >= min_class_count]

    # Filter the data to include only those classes
    X_filtered = [x for x, cls in zip(X, y) if cls in valid_classes]
    y_filtered = [cls for cls in y if cls in valid_classes]

    # Stratified sampling to maintain the class distribution
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_index, sample_index in split.split(X_filtered, y_filtered):
        X_new = [X_filtered[i] for i in sample_index]
        y_new = [y_filtered[i] for i in sample_index]

    return X_new, y_new

def filter_classes(X, y, threshold=100):
    # Count the occurrences of each class in y
    class_counts = Counter(y)

    # Find the classes that meet the threshold
    valid_classes = {cls for cls, count in class_counts.items() if count >= threshold}

    # Filter the instances of X and y based on the valid classes
    X_filtered = [x for x, cls in zip(X, y) if cls in valid_classes]
    y_filtered = [cls for cls in y if cls in valid_classes]

    return np.array(X_filtered), np.array(y_filtered)

# Filter classes with fewer than 100 instances
X, y = filter_classes(X, y)

# Use only 10% of the data
X, y = stratified_split(X, y)

In [7]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import numpy as np

def balance_data(X, y, target_samples=50000, seed=42):
    # First, apply under-sampling to reduce the majority class
    under_sampler = RandomUnderSampler(sampling_strategy={0: target_samples}, random_state=seed)
    X_under, y_under = under_sampler.fit_resample(X, y)

    # Next, apply SMOTE to increase the minority classes
    smote = SMOTE(random_state=seed)
    new_X, new_y = smote.fit_resample(X_under, y_under)

    return new_X, new_y


In [8]:
# Splitting the Data
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)

# Print class distribution before balancing
unique_before, counts_before = np.unique(y, return_counts=True)
print("Class distribution before balancing:", dict(zip(unique_before, counts_before)))

# Balance the data
X_train, y_train = balance_data(X_train, y_train, target_samples=25000, seed=42)

# Print class distribution after balancing
unique_after, counts_after = np.unique(y_train, return_counts=True)
print("Class distribution after balancing:", dict(zip(unique_after, counts_after)))


# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

Class distribution before balancing: {0: 227310, 2: 12803, 3: 1029, 4: 23107, 5: 550, 6: 579, 7: 794, 10: 15893, 11: 590}
Class distribution after balancing: {0: 25000, 2: 25000, 3: 25000, 4: 25000, 5: 25000, 6: 25000, 7: 25000, 10: 25000, 11: 25000}


<ipython-input-8-fddd3811cad7>:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)


In [9]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)


In [10]:
from sklearn.metrics import confusion_matrix

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=4, num_layers=4, dim_feedforward=128, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add a fake batch dimension for the transformer
        x = self.transformer(x, x)  # Encoder-Decoder Self-Attention
        x = x.squeeze(1)
        x = self.fc(x)
        return x


def evaluate_model(model, data_loader):
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for data, labels in data_loader:
            data = data.cuda()
            labels = labels.cuda()

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')

    return accuracy, f1, precision, recall


In [11]:
from sklearn.metrics import balanced_accuracy_score, classification_report

def evaluate_model(model, data_loader) -> tuple:
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for data, labels in data_loader:
            data = data.cuda()
            labels = labels.cuda()

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(true_labels, predicted_labels)
    balanced_accuracy = balanced_accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')

    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')
    macro_precision = precision_score(true_labels, predicted_labels, average='macro')
    macro_recall = recall_score(true_labels, predicted_labels, average='macro')

    print("Classification Report : \n", classification_report(true_labels, predicted_labels))

    return accuracy, balanced_accuracy, f1, precision, recall, macro_f1, macro_precision, macro_recall

In [20]:
from torch.optim import Adam, SGD, Adagrad, RMSprop, AdamW

import numpy as np

def train_model(optimizer_class,y):
    # Convert X_train to a NumPy array if it's a list
    X_train_array = np.array(X_train)
    input_dim = X_train_array.shape[1]
    unique_labels = np.unique(y)
    label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
    y = np.vectorize(label_mapping.get)(y)
    num_classes = len(unique_labels)

    model = TransformerModel(input_dim, num_classes, num_layers=6)  # Adjust as needed
    model = model.cpu()

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_class(model.parameters(), lr=0.001)

    # Dataloader with new batch size
    batch_size = 1024
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for i, (data, labels) in enumerate(train_loader):
            data = data.cpu()
            labels = labels.cpu()

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Optimizer: {optimizer.__class__.__name__}, Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}')

    # Evaluate model
    # Ensure evaluate_model function is defined
    acc, balanced_acc, f1, prec, rec, macro_f1, macro_prec, macro_rec = evaluate_model(model, DataLoader(test_dataset, batch_size=batch_size))
    print(f'Optimizer: {optimizer.__class__.__name__}, Accuracy: {acc:.4f}, Balanced Accuracy: {balanced_acc:.4f}, F1: {f1:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, Macro F1: {macro_f1:.4f}, Macro Precision: {macro_prec:.4f}, Macro Recall: {macro_rec:.4f}')

    return acc, balanced_acc, f1, prec, rec, macro_f1, macro_prec, macro_rec


In [21]:
# Create a dictionary of optimizers
optimizers_dict = {
    'Adam': Adam,
    'SGD': SGD,
    'Adagrad': Adagrad,
    'RMSprop': RMSprop,
    'AdamW': AdamW
}

# Lists to store metrics for each model configuration
accuracies = []
balanced_accuracies = []
f1_scores = []
precisions = []
recalls = []
macro_f1_scores = []
macro_precisions = []
macro_recalls = []

for opt_name, opt in optimizers_dict.items():
    acc, balanced_acc, f1, prec, rec, macro_f1, macro_prec, macro_rec = train_model(opt, y_train)
    accuracies.append(acc)
    balanced_accuracies.append(balanced_acc)
    f1_scores.append(f1)
    precisions.append(prec)
    recalls.append(rec)
    macro_f1_scores.append(macro_f1)
    macro_precisions.append(macro_prec)
    macro_recalls.append(macro_rec)


IndexError: ignored

In [ ]:
unique_labels = np.unique(y)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
y = np.vectorize(label_mapping.get)(y)
num_classes = len(unique_labels) # or 12 if the labels range from 0 to 11


In [ ]:
print("Number of unique labels:", len(np.unique(y)))
print("Max label:", np.max(y))
print("Num classes:", num_classes)
assert np.max(y) < num_classes, "Labels are out of range!"



In [ ]:
# Plotting the metrics
import matplotlib.pyplot as plt
import numpy as np

# Create an index for each tick position
x_indexes = np.arange(len(optimizers_dict.keys()))
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 6))

# Generate bar graphs
rects1 = ax.bar(x_indexes - 2 * width, accuracies, width, label='Accuracy')
rects2 = ax.bar(x_indexes - width, f1_scores, width, label='F1 Score')
rects3 = ax.bar(x_indexes, precisions, width, label='Precision')
rects4 = ax.bar(x_indexes + width, recalls, width, label='Recall')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Optimizers')
ax.set_ylabel('Scores')
ax.set_title('Scores by Optimizers')
ax.set_xticks(x_indexes)
ax.set_xticklabels(optimizers_dict.keys())
ax.legend()

# Attach a text label above each bar in *rects*, displaying its height
for rects in [rects1, rects2, rects3, rects4]:
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{:.2f}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

fig.tight_layout()

plt.show()


In [ ]:
print("optimizers:", optimizers_dict )
print("accuracies:", accuracies)
print("f1_scores:", f1_scores)
print("precisions:", precisions)
print("recalls:", recalls)